In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#!pip category_encoders
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import scipy.stats as stats
from sklearn.feature_extraction.text import TfidfVectorizer
#import category_encoders as ce
import warnings
import gc
import psutil 
warnings.filterwarnings('ignore')
%matplotlib inline

os.chdir('/content/drive/My Drive/ML/FinalProject')
#os.chdir('dataset')
!ls

 dataset			     vef_2sigma.ipynb
'Two Sigma Revised.pptx'	     vef_2sigma_modeling_CNN.ipynb
 vef_2sigma_feature_engineer.ipynb   vef_2sigma_modeling.ipynb


In [0]:
#!gunzip market_train_uni1_v1.csv.gz

In [0]:
#!gunzip news_market_train_uni1_v1.csv.gz

In [0]:
!ls dataset

market_train_uni1_v1.csv  news_market_train_uni1_v1.csv


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
market_dt = pd.read_csv('dataset/market_train_uni1_v1.csv')
market_null_list = market_dt.columns[market_dt.isna().any()].tolist()
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp = imp.fit(market_dt[market_null_list])
market_dt[market_null_list] = imp.transform(market_dt[market_null_list])


In [0]:
news_dt = pd.read_csv('dataset/news_market_train_uni1_v1.csv')

In [0]:
def label_cat(df, col):
    return df[col].astype('category').cat.as_ordered()

def bin_encode(df, cols_to_bin):
    ce_bin = ce.BinaryEncoder(cols = cols_to_bin)
    return ce_bin.fit_transform(df)

# Find all features with NaN's
def get_null_features(df):
    return df.columns[df.isna().any()].tolist()

# Returns either month, day or year of date -> expedites feature engineering 
def get_date_feature(df, col, date_type):
    if (date_type == 'year'):
        return pd.to_datetime(df[col]).dt.year
    elif (date_type == 'month'):
        return pd.to_datetime(df[col]).dt.month
    elif (date_type == 'quarter'):
        return pd.to_datetime(df[col]).dt.quarter
    else:
        return pd.to_datetime(df[col]).dt.dayofweek

# Helper function to print accuracy
def print_accuracy(model, x, y, x_val, y_val):
    print("Training Accuracy Score: ", accuracy_score(model.predict(x), y))
    print("Validation Accuracy Score: ", accuracy_score(model.predict(x_val), y_val))
    
# Helper function to get feature importances
def get_feature_imp(model, x):
    return pd.DataFrame(model.feature_importances_,
                                   index = x.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

# Calculate the RSI
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

# 80%, 20%
def temploral_split(df, labels):
    X_train, X_val = np.split(df, [int(.8*len(df))])
    y_t, y_v = np.split(labels, [int(.8*len(labels))])
    
    y_train = y_t > 0    
    y_val = y_v > 0
    
    return [X_train, y_train, X_val, y_val]

# Join 2 dataframes
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

# Random search for temporal data
def random_search(model, param_grid, X_train, y_train, X_val, y_val):
    best_acc = 0
    for i in range(10):
        params = {k: np.random.choice(v) for k, v in param_grid.items()}
        params['n_jobs'] = -1
        new_model = model(**params)
        new_model.fit(X_train, y_train)
        acc = accuracy_score(new_model.predict(X_val), y_val)

        if acc > best_acc or best_acc == 0:
            best_acc = acc
            best_params = params
    print(f'Best Score: {best_acc}')
    print(f'Best Paramas: {best_params}')
    return best_params, best_acc


In [0]:
# Prep market data
# input: market dataframe
# output: preprocessed and feature engineered dataframe
def prep_market(df):
    
    
    #df['time'] = df.time.dt.strftime("%Y%m%d").astype(int)
    
    # Create year, month, day features
    df['month'] = get_date_feature(df, 'time', 'month')
    df['day'] = get_date_feature(df, 'time', 'day')
    df['quarter'] = get_date_feature(df, 'time', 'quarter')
    
    # Some feature engineering -> moving averages
    for n in [14, 30, 50, 200]:
        # Create the moving averages
        df['close_ma' + str(n)] = df['close'].rolling(window=n).mean()
    
    # Create RSI -> only 14 was useful during feature exploration
    df['rsi14'] = RSI(df['close'], 14)
    
    # 10 day pct change in volume
    df['vol_pct_change'] = df['volume'].pct_change()
    df['vol_pct_10'] = df['vol_pct_change'].rolling(window=10).mean()
    
    # Log Returns
    df['log_ret'] = np.log(df['close']).diff()

    # Momentum
    df['mom1'] = df['close'].pct_change(periods=1)
    df['mom2'] = df['close'].pct_change(periods=2)
    df['mom3'] = df['close'].pct_change(periods=3)
    df['mom4'] = df['close'].pct_change(periods=4)
    df['mom5'] = df['close'].pct_change(periods=5)
    df['mom10'] = df['close'].pct_change(periods=10)

    # Volatility
    df['volatility_50'] = df['log_ret'].rolling(window=50, min_periods=50, center=False).std()
    df['volatility_31'] = df['log_ret'].rolling(window=31, min_periods=31, center=False).std()
    df['volatility_15'] = df['log_ret'].rolling(window=15, min_periods=15, center=False).std()
    window_autocorr = 50

    #market_dt['autocorr_1'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
    #market_dt['autocorr_2'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
    #market_dt['autocorr_3'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
    #market_dt['autocorr_4'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
    #market_dt['autocorr_5'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

    # drop 200 rows -> this is because of the moving average calculations
    df.dropna(inplace=True)
    _ = gc.collect()
    
    # These were identified in part 2 -> returnsOpenNextMktres10 is the dependant var
    train_cols = ['returnsClosePrevRaw10',
                  'returnsClosePrevMktres10',  
                  'close',
                  'rsi14', 
                  'assetCode',
                  'month', 
                  'close_ma14',
                  'close_ma200',
                  'close_ma30',
                  'volume',
                  'close_ma50',
                  'returnsClosePrevMktres1', 
                  'returnsClosePrevRaw1',
                  'returnsOpenPrevRaw1',
                  'vol_pct_10',
                  'day',
                  'quarter',
                  'time',
                  'mom1',
                  'mom2',
                  'mom3',
                  'mom4',
                  'mom5',
                  'mom10',
                  'volatility_50',
                  'volatility_31',
                  'volatility_15'
                   ]
    
    # This is for the final trainin set contidion
    if 'returnsOpenNextMktres10' in df.columns:
        train_cols = train_cols + ['returnsOpenNextMktres10']
    
    df = df[train_cols]
    
    return df

# Prep news data
# input: news dataframe
# output: preprocessed and feature engineered dataframe
def prep_news(df):
    
    drop_list = [
        'audiences', 'subjects', 'assetName',
        'firstCreated', 'sourceTimestamp',
    ]
    df.drop(drop_list, axis=1, inplace=True)
    _ = gc.collect()
    
    # convert the date
    #df['time'] = df.time.dt.strftime("%Y%m%d").astype(int)
    
    # convert the assets codes to a usable format
    df['assetCode'] = df['assetCodes'].map(lambda x: list(eval(x))[0])
    
    # encode provider
    df['provider'] = label_cat(df, 'provider').cat.codes

    # determine the proportion of the news item discussing the asset
    df['coverage'] = df['sentimentWordCount'] / df['wordCount']

    # relative position of the first mention in the item
    df['position'] = df['firstMentionSentence'] / df['sentenceCount']
    
    # Drop some unnecessary news features 
    droplist = ['takeSequence','headlineTag',
                'assetCodes', 'headline', 'marketCommentary']

    df.drop(droplist, axis=1, inplace=True)
    _ = gc.collect()
    
    # combine multiple news reports for same assets on same day
    newsgp = df.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    
    return newsgp

In [0]:
# Function for all data processing
# input: market and news dataframe
# output: preprocessed, feature engineered, grouped and joined dataframe
def prep_data(market_train_df, news_train_df):
    market_train = prep_market(market_train_df)
    news_train = prep_news(news_train_df)
    
    joined = join_df(market_train, news_train, ['time', 'assetCode'], ['time', 'assetCode'])

    # many assets that will have many days without news data
    joined[news_train.columns[2:].values] = joined[news_train.columns[2:].values].fillna(value=0)
    joined.drop(['time'], axis=1, inplace=True)
    joined['assetCodeT'] = label_cat(joined, 'assetCode').cat.codes
    
    del market_train
    del news_train
    _ = gc.collect()
    
    return joined

In [0]:
ts_df = prep_data(market_dt, news_dt)
#market_train = prep_market(market_dt) 

In [13]:
ts_df.head()

,returnsClosePrevRaw10,returnsClosePrevMktres10,close,rsi14,assetCode,month,close_ma14,close_ma200,close_ma30,volume,close_ma50,returnsClosePrevMktres1,returnsClosePrevRaw1,returnsOpenPrevRaw1,vol_pct_10,day,quarter,mom1,mom2,mom3,mom4,mom5,mom10,volatility_50,volatility_31,volatility_15,returnsOpenNextMktres10,urgency,provider,bodySize,companyCount,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,coverage,position,assetCodeT
0,-0.031188,0.001085,19.57,46.742510,CMVT.OQ,2,55.325000,46.73155,50.548000,17312577.0,46.2914,-0.000101,0.004105,0.017801,2.254785,3,1,0.157988,-0.856861,-0.535154,-0.544035,-0.296549,-0.712671,0.800958,0.880357,1.056708,0.032696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,504
1,0.040274,0.001085,60.70,51.105459,CMX.N,2,50.047143,46.87410,51.618333,9916700.0,47.0054,-0.000101,-0.009141,0.003462,2.272481,3,1,2.101686,2.591716,-0.556027,0.441805,0.414259,0.762997,0.811842,0.890360,1.087940,0.059179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,505
2,-0.021028,0.001085,12.57,46.323133,CNE.N,2,48.987857,46.74940,49.574000,1443200.0,46.7400,-0.000101,-0.003962,0.004754,2.248443,3,1,-0.792916,-0.357690,-0.256213,-0.908060,-0.701425,-0.775255,0.842482,0.935526,0.987081,-0.008823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,509
3,0.037333,0.001085,46.68,49.901210,CNI.N,2,49.375000,46.55910,50.451667,1781400.0,46.2400,-0.000101,0.021668,0.032141,1.919634,3,1,2.713604,-0.230972,1.385284,1.762130,-0.658572,-0.301407,0.850763,0.950276,0.970431,0.007802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,513
4,0.059964,0.001085,17.50,47.014002,CNP.N,2,45.760000,46.55650,49.165000,2748800.0,45.7492,-0.000101,0.013905,-0.010876,2.025298,3,1,-0.625107,0.392204,-0.711697,-0.105774,0.035503,-0.708236,0.858716,0.938016,0.998184,0.037972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,517


In [0]:
ts_df.head()

,returnsClosePrevRaw10,returnsClosePrevMktres10,close,rsi14,assetCode,month,close_ma14,close_ma200,close_ma30,volume,close_ma50,returnsClosePrevMktres1,returnsClosePrevRaw1,returnsOpenPrevRaw1,vol_pct_10,day,quarter,returnsOpenNextMktres10,urgency,provider,bodySize,companyCount,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,coverage,position,assetCodeT
0,-0.031188,0.001085,19.57,46.742510,CMVT.OQ,2,55.325000,46.73155,50.548000,17312577.0,46.2914,-0.000101,0.004105,0.017801,2.254785,3,1,0.032696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,504
1,0.040274,0.001085,60.70,51.105459,CMX.N,2,50.047143,46.87410,51.618333,9916700.0,47.0054,-0.000101,-0.009141,0.003462,2.272481,3,1,0.059179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,505
2,-0.021028,0.001085,12.57,46.323133,CNE.N,2,48.987857,46.74940,49.574000,1443200.0,46.7400,-0.000101,-0.003962,0.004754,2.248443,3,1,-0.008823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,509
3,0.037333,0.001085,46.68,49.901210,CNI.N,2,49.375000,46.55910,50.451667,1781400.0,46.2400,-0.000101,0.021668,0.032141,1.919634,3,1,0.007802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,513
4,0.059964,0.001085,17.50,47.014002,CNP.N,2,45.760000,46.55650,49.165000,2748800.0,45.7492,-0.000101,0.013905,-0.010876,2.025298,3,1,0.037972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,517


In [0]:
def temploral_split(df, label):
    df.drop(['assetCode'],axis=1, inplace=True)
    train, validate = np.split(df, [int(.8*len(df))])
    
    X_train = train.loc[:, ~train.columns.isin([label])]
    y_train = train[label] > 0
    
    X_val = validate.loc[:, ~validate.columns.isin([label])]
    y_val = validate[label] > 0
    
    return [X_train, y_train, X_val, y_val]
def print_accuracy(model, x, y, x_val, y_val):
    print("Training Accuracy Score: ", accuracy_score(model.predict(x), y))
    print("Validation Accuracy Score: ", accuracy_score(model.predict(x_val), y_val))

In [15]:

X_train_sample, y_train_sample, X_val_sample, y_val_sample = temploral_split(ts_df.iloc[-2000000:], 'returnsOpenNextMktres10')
print(f'X_train_sample: {X_train_sample.shape}')
print(f'X_val_sample: {X_val_sample.shape}')


X_train_sample: (1600000, 51)
X_val_sample: (400000, 51)


In [0]:
from sklearn.model_selection import GridSearchCV
MODEL = 'RF'
parameters = {'max_depth':[ 3 , 5, 7],
              'n_estimators':[10, 25, 50, 100],
              'random_state':[42]}
def perform_grid_search(X_data, y_data):
    if MODEL == 'RF':
        rf = RandomForestClassifier(criterion='entropy')
    else:
        rf = GradientBoostingClassifier()
    
    clf = GridSearchCV(rf, parameters, cv=4, scoring='roc_auc', n_jobs=3)
    
    clf.fit(X_data, y_data)
    
    print(clf.cv_results_['mean_test_score'])
    
    return clf.best_params_['n_estimators'], clf.best_params_['max_depth']

In [0]:

n_estimator, depth = perform_grid_search(X_train_sample, y_train_sample)
c_random_state = 42
print(n_estimator, depth, c_random_state)

[0.53561623 0.53635244 0.5385174  0.53833496 0.53473664 0.5376032
 0.53965403 0.54009973 0.53436889 0.53703035 0.53918291 0.54020553]
100 7 42


In [16]:
rfc = ensemble.RandomForestClassifier(n_estimators=200, criterion='gini',max_depth = 10, max_features=0.6, n_jobs=-1, random_state=23)
%time rfc.fit(X_train_sample, y_train_sample)


CPU times: user 2h 8min 8s, sys: 1.86 s, total: 2h 8min 9s
Wall time: 32min 20s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features=0.6, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=23, verbose=0,
                       warm_start=False)

In [17]:
print_accuracy(rfc, X_train_sample, y_train_sample, X_val_sample, y_val_sample)


Training Accuracy Score:  0.574569375
Validation Accuracy Score:  0.54268


In [18]:
import lightgbm as lgb
params = {'learning_rate': 0.01, 'max_depth': 12, 'boosting': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'is_training_metric': True, 'seed': 42}
model = lgb.train(params, train_set=lgb.Dataset(X_train_sample, label=y_train_sample), num_boost_round=2000,
                  valid_sets=[lgb.Dataset(X_val_sample, label=y_val_sample), lgb.Dataset(X_val_sample, label=y_val_sample)],
                  verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.558188	valid_1's auc: 0.558188
[200]	valid_0's auc: 0.559924	valid_1's auc: 0.559924
[300]	valid_0's auc: 0.560723	valid_1's auc: 0.560723
[400]	valid_0's auc: 0.56138	valid_1's auc: 0.56138
[500]	valid_0's auc: 0.561495	valid_1's auc: 0.561495
Early stopping, best iteration is:
[482]	valid_0's auc: 0.561551	valid_1's auc: 0.561551


In [0]:
import xgboost as xgb
from xgboost import XGBClassifier
model_xgb = xgb.XGBClassifier(n_estimators = 400,max_depth=4)
model_xgb.fit(X_train_sample, y_train_sample)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Training Accuracy Score:  0.569510625
Validation Accuracy Score:  0.54483


In [0]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
N_THREADS = -1
RANDOM_SEED = 23
model_extree = ExtraTreesClassifier(n_estimators=75, max_depth=8, min_samples_split=30, min_samples_leaf=16, n_jobs=N_THREADS, random_state=RANDOM_SEED)
model_extree.fit(X_train_sample, y_train_sample)


NameError: ignored

In [0]:
print_accuracy(model_extree, X_train_sample, y_train_sample, X_val_sample, y_val_sample)

Training Accuracy Score:  0.52359125
Validation Accuracy Score:  0.5120525


In [0]:
from sklearn.linear_model import RidgeClassifier
model_lr0 = RidgeClassifier(fit_intercept=False)
model_lr0.fit(X_train_sample, y_train_sample)

model_lr1 = RidgeClassifier(fit_intercept=False)
model_lr1.fit(X_train_sample, y_train_sample)

model_lr2 = RidgeClassifier(fit_intercept=False)
model_lr2.fit(X_train_sample, y_train_sample)


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=False,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [0]:
print_accuracy(model_lr1, X_train_sample, y_train_sample, X_val_sample, y_val_sample)

Training Accuracy Score:  0.543545
Validation Accuracy Score:  0.5375


In [0]:
def print_mix_accuracy(dic_model,X,y):
    predict = np.zeros(len(X))
    for model , weight in dic_model.items():
        predict = np.add(predict, weight* model.predict(X))
    predict = predict > 0.5
    predict = 1*predict
    print (accuracy_score(predict, y))

In [0]:
dic_model  = {model_extree: 0.6, model_lr0: 0.22, model_lr1: 0.03, model_lr2: 0.15}

In [0]:
print_mix_accuracy(dic_model,X_train_sample, y_train_sample)

0.52359125
